# Hybrid Demucs from Colab

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-mb8p8exy/demucs_f601425b21e142a39e8507e75debd3d1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-mb8p8exy/demucs_f601425b21e142a39e8507e75debd3d1
  Resolved https://github.com/facebookresearch/demucs to commit e7367cfae80538cbddc96e093249c3c81cb88d31
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 KB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.4/425.4 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 KB 7.3 MB/s eta 0:00:00
  Preparing m

In [ ]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/drive/MyDrive/GTZAN/genres_original/hiphop'
out_path = '/content/drive/MyDrive/GTZAN/demux/hiphopx'

In [ ]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
/content/drive/MyDrive/GTZAN/genres_original/country/country.00000.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00001.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00005.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00013.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00003.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00004.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00009.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00014.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00006.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00011.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00008.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00010.wav
/content/drive/MyDrive/GTZAN/genres_original/country/country.00002.wav
/content/drive/MyDrive/GTZAN/genres_original/cou

100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:03<00:00, 11.38seconds/s]


Separating track /content/drive/MyDrive/GTZAN/genres_original/country/country.00001.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 21.67seconds/s]


Separating track /content/drive/MyDrive/GTZAN/genres_original/country/country.00005.wav


...

100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 20.87seconds/s]


Separating track /content/drive/MyDrive/GTZAN/genres_original/country/country.00093.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 20.82seconds/s]


Separating track /content/drive/MyDrive/GTZAN/genres_original/country/country.00099.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 21.01seconds/s]


In [ ]:
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
files.download('./separated.zip')